In [ ]:
import scipy.stats as sps
from hdfs3 import HDFileSystem
import pandas as pd
import os


In [12]:
hdfs = HDFileSystem(user='ayu')

def read_hdfs(path):
    files = hdfs.glob(path)
    dfs = []
    for file in files:
        with hdfs.open(file) as f:
            dfs.append(pd.read_parquet(f))
    return pd.concat(dfs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Can not find the shared library: libhdfs3.so
See installation instructions at http://hdfs3.readthedocs.io/en/latest/install.html
Traceback (most recent call last):
  File "/usr/lib/python2.7/site-packages/hdfs3/core.py", line 88, in __init__
    self.connect()
  File "/usr/lib/python2.7/site-packages/hdfs3/core.py", line 106, in connect
    get_lib()
  File "/usr/lib/python2.7/site-packages/hdfs3/core.py", line 670, in get_lib
    from .lib import _lib as l
  File "/usr/lib/python2.7/site-packages/hdfs3/lib.py", line 24, in <module>
    raise ImportError("Can not find the shared library: libhdfs3.so\n"
ImportError: Can not find the shared library: libhdfs3.so
See installation instructions at http://hdfs3.readthedocs.io/en/latest/install.html



In [11]:
# Delay averages
default = read_hdfs('/user/%s/final/parquet/default/*.parquet' % username)
t_type_line = read_hdfs('/user/%s/final/parquet/t_type_line/*.parquet' % username)
t_type_line_hour = read_hdfs('/user/%s/final/parquet/t_type_line_hour/*.parquet' % username)
t_type_line_hour_stop = read_hdfs('/user/%s/final/parquet/t_type_line_hour_stop/*.parquet' % username)

# Dicts to speedup average delay lookup times
t_type_line_dict = t_type_line.set_index(['transport_type', 'line_name']).to_dict()['avg_delay']
t_type_line_hour_dict = t_type_line_hour.set_index(['transport_type', 'line_name', 'hour']).to_dict()['avg_delay']
t_type_line_hour_stop_dict = t_type_line_hour_stop.set_index(['transport_type', 'line_name', 'hour', 'stop_id']).to_dict()['avg_delay']
default_delay = default.values[0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
global name 'hdfs' is not defined
Traceback (most recent call last):
  File "<stdin>", line 2, in read_hdfs
NameError: global name 'hdfs' is not defined



In [ ]:
def predict_delay_probability(max_delay, t_type, line_name, hour, stop_id):
    """
    t_type: transport_type(PRODUCT_ID)
    line_name: line_name(linien_text)
    hour: hour for arrival_scheduled
    stop_id: stop station id
    """
    prob = 1.0
    if (t_type, line_name, hour, stop_id) in t_type_line_hour_stop_dict:
        delay_pred = t_type_line_hour_stop_dict[(t_type, line_name, hour, stop_id)]
    elif (t_type, line_name, hour) in t_type_line_hour_dict:
        delay_pred = t_type_line_hour_dict[(t_type, line_name, hour)]
    elif (t_type, line_name) in t_type_line_dict:
        delay_pred = t_type_line_hour_dict[(t_type, line_name)]
    else:
        delay_pred = default_delay
                
    if lambda_ != 0:
        #prob *= exponential_cdf(delay_pred, max_delay * 60)
        prob = sps.expon.cdf(max_delay, scale=1/delay_pred)
        
    return prob
